In [1]:
import itertools
from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision

In [2]:
trans = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                            transforms.Resize((14,28)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, ),(0.5, ))])

data = torchvision.datasets.ImageFolder(root = './data',
                                       transform = trans)

test = torchvision.datasets.ImageFolder(root = './val',
                                       transform = trans)

data.classes
test.classes

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'dot']

In [3]:
batch_size = 50
trn_loader = DataLoader(data,
                        batch_size=batch_size,
                       shuffle = True,
                       drop_last=True)

val_loader = DataLoader(test,
                        shuffle = False)


# dataiter = iter(trn_loader)
# images, labels = dataiter.next()
# print(labels[0])

In [4]:
# construct model on cuda if available
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cpu


In [5]:
class CNNClassifier(nn.Module):

    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNNClassifier, self).__init__()
        conv1 = nn.Conv2d(1, 6, 5, 1)  # 6@24*24
        # activation ReLU
        pool1 = nn.MaxPool2d(2)  # 6@12*12
        conv2 = nn.Conv2d(6, 16, 5, 1)  # 16@8*8
        # activation ReLU
        pool2 = nn.MaxPool2d(2)  # 16@4*4

        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2
        )

        fc1 = nn.Linear(64, 32)
        # activation ReLU
        fc2 = nn.Linear(32, 16)
        # activation ReLU
        fc3 = nn.Linear(16, 11)

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.ReLU(),
            fc3
        )

        # gpu로 할당
        if use_cuda:
            self.conv_module = self.conv_module.cuda()
            self.fc_module = self.fc_module.cuda()

    def forward(self, x):
        out = self.conv_module(x)  # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]:  # 16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return F.softmax(out, dim=1)

cnn = CNNClassifier()

In [6]:
# loss
criterion = nn.CrossEntropyLoss()
# backpropagation method
learning_rate = 1e-3
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
# hyper-parameters
num_epochs = 2
num_batches = len(trn_loader)

trn_loss_list = []
val_loss_list = []
for epoch in range(num_epochs):
    trn_loss = 0.0
    for i, data in enumerate(trn_loader):
        x, label = data
#         if use_cuda:
#             x = x.cuda()
#             label = label.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        model_output = cnn(x)
        # calculate loss
        loss = criterion(model_output, label)
        # back propagation
        loss.backward()
        # weight update
        optimizer.step()

        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del model_output

In [8]:
import statistics
acc = []
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
#     X_test, Y_test = val_loader
    for i, data in enumerate(val_loader):
        x_test, Y_test = data

        prediction = cnn(x_test)
#         print(prediction)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        acc.append(accuracy.item())
    print(statistics.mean(acc))
#         print('Accuracy%d:'%i, accuracy.item())

0.16494845360824742


In [7]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
#     X_test, Y_test = val_loader
    for i, data in enumerate(val_loader):
        x_test, Y_test = data

        prediction = cnn(x_test)
        print(prediction)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

tensor([[0.0769, 0.0379, 0.1192, 0.0938, 0.0335, 0.1212, 0.1209, 0.1096, 0.1137,
         0.0946, 0.0787]])
Accuracy: 0.0
tensor([[0.0686, 0.0284, 0.1144, 0.0906, 0.0220, 0.1220, 0.1429, 0.1233, 0.1304,
         0.0688, 0.0884]])
Accuracy: 0.0
tensor([[0.0098, 0.0009, 0.0155, 0.0095, 0.0016, 0.0791, 0.6147, 0.0684, 0.0284,
         0.0093, 0.1628]])
Accuracy: 0.0
tensor([[0.0351, 0.0086, 0.0567, 0.0529, 0.0039, 0.0718, 0.2572, 0.1652, 0.1212,
         0.0175, 0.2098]])
Accuracy: 0.0
tensor([[0.0329, 0.0195, 0.2631, 0.0313, 0.0017, 0.0185, 0.0248, 0.3943, 0.0190,
         0.0369, 0.1580]])
Accuracy: 0.0
tensor([[0.0526, 0.0686, 0.2188, 0.0910, 0.0058, 0.0161, 0.0220, 0.2667, 0.0312,
         0.0653, 0.1618]])
Accuracy: 0.0
tensor([[0.0486, 0.0198, 0.3064, 0.0365, 0.0052, 0.0544, 0.0614, 0.2537, 0.0388,
         0.0542, 0.1211]])
Accuracy: 0.0
tensor([[0.0307, 0.0376, 0.1672, 0.0484, 0.0015, 0.0069, 0.0113, 0.4393, 0.0119,
         0.0367, 0.2085]])
Accuracy: 0.0
tensor([[1.5672e-02, 3.1

In [ ]:
# prediction = cnn(images)
# print(prediction)

for data in val_loader:
    x_test, Y_test = data
    
    print('Label: ', Y_test.item())
    single_prediction = cnn(x_test)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())